In [1]:
import Annotator
import gff_helpers
import gffutils
import region_helpers as r
import pybedtools
import os
from tqdm import tnrange, tqdm_notebook

In [2]:
db_file = '/projects/ps-yeolab/genomes/hg19/gencode_v19/gencode.v19.annotation.gtf.db'
# db_file = '/projects/ps-yeolab3/bay001/annotations/gencode.vM10.annotation.db'

In [3]:
a = Annotator.Annotator(db_file, chroms=['chr7'])

In [4]:
d = '/home/bay001/projects/codebase/bfx/pyscripts/data/'
bed = pybedtools.BedTool(os.path.join(d,'WTV5_02_01.basedon_WTV5_02_01.peaks.l2inputnormnew.bed.compressed.chr7.bed'))

In [7]:
features = {}
with open('/home/bay001/test.bed','w') as f:
    progress = tnrange(len(bed))
    for interval in bed:
        fs = a.get_all_overlapping_features_from_query(interval.chrom, interval.start, interval.end, interval.strand)
        features['{}:{}-{}'.format(interval.chrom, interval.start, interval.end)] = fs
        f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(
            interval.chrom,
            interval.start,
            interval.end,
            interval.name,
            interval.score,
            interval.strand,
            a.prioritize(fs, v=False)
        ))
        if interval.start == 65440934:
            print(interval.end)
            a.prioritize(fs, v=True)
        progress.update(1)

Widget Javascript not detected.  It may not be installed or enabled properly.



65440963
ENSG00000169919.12
('UTR', ['ENST00000446111.1'], ['ENSG00000169919.12'], 65440839, 65441189)
('gene', ['ENSG00000169919.12'], ['ENSG00000169919.12'], 65425671, 65447301)
('exon', ['ENST00000466883.1'], ['ENSG00000169919.12'], 65439282, 65441147)
('exon', ['ENST00000446111.1'], ['ENSG00000169919.12'], 65440839, 65441189)
('transcript', ['ENST00000304895.4'], ['ENSG00000169919.12'], 65425671, 65447301)
('transcript', ['ENST00000466883.1'], ['ENSG00000169919.12'], 65425675, 65441147)
('transcript', ['ENST00000447929.1'], ['ENSG00000169919.12'], 65425685, 65447183)
('transcript', ['ENST00000430730.1'], ['ENSG00000169919.12'], 65425685, 65447202)
('transcript', ['ENST00000421103.1'], ['ENSG00000169919.12'], 65425688, 65447198)
('transcript', ['ENST00000345660.6'], ['ENSG00000169919.12'], 65425688, 65447198)
('transcript', ['ENST00000465785.1'], ['ENSG00000169919.12'], 65439281, 65444495)
('transcript', ['ENST00000479038.1'], ['ENSG00000169919.12'], 65439344, 65445398)
('transcrip

In [6]:
chr7:65440934-65440963

SyntaxError: invalid syntax (<ipython-input-6-2f4194e568f6>, line 1)

In [ ]:
len(a.get_all_overlapping_features_from_query(u'chr7',98479373,98479416))

In [ ]:
features['chr7:104742399-104742504']

In [ ]:
r.classify(features['chr7:104742399-104742504'])

In [ ]:
genes = gene_dict(features['chr7:104742399-104742504'])

In [ ]:
def transcript_dict(features, priority=PRIORITY):
    """
    Groups each transcript feature into genes.
    """
    transcripts = defaultdict(list)
    for feature in features:
        for transcript_id in feature.attributes['transcript_id']:
            transcripts[transcript_id].append(feature)
    for transcript, features in transcripts.iteritems():
        features.sort(key=lambda x: priority.index(x.featuretype.upper()))
    return transcripts

In [ ]:
from collections import defaultdict

# PRIORITY = ['UTR', 'CDS', 'utr3', 'EXON', 'GENE', 'TRANSCRIPT']
PRIORITY = ['CDS', 'UTR', 'GENE', 'EXON', 'TRANSCRIPT', 'START_CODON', 'STOP_CODON']

def prioritize(features, priority=PRIORITY):
    """
    # Groups each transcript feature into genes.
    """
    hi = [] # contains highest priority genic feature for each gene
    genes = defaultdict(list)
    for feature in features:
        for gene_id in feature.attributes['gene_id']:
            genes[gene_id].append(feature)
    for gene, features in genes.iteritems():
        features.sort(key=lambda x: priority.index(x.featuretype.upper()))
    for gene_id, overlapping_features in genes.iteritems():
        
        hi.append('{}|{}'.format(overlapping_features[0].featuretype.replace('gene','INTRON'), gene_id))
    return hi

In [ ]:
genes